In [ ]:
# Install the Hugging Face transformers library
!pip install transformers -q

# Import necessary libraries
import pandas as pd
from transformers import pipeline

print("Libraries installed and imported successfully!")

Libraries installed and imported successfully!


In [ ]:
# The filename of your refined dataset
file_name = 'zomato_refined.csv'

try:
    # Load the refined dataset from the CSV file
    df = pd.read_csv(file_name)

    # Display the first few rows to confirm it's loaded and looks correct
    print(f"✅ Successfully loaded '{file_name}'. Here is a preview:")
    print(df.head())

except FileNotFoundError:
    print(f"❌ ERROR: The file '{file_name}' was not found.")
    print("Please make sure you have uploaded your refined dataset and the filename matches exactly.")

✅ Successfully loaded 'zomato_refined.csv'. Here is a preview:
                    name                                            address  \
0                  Jalsa  942, 21st Main Road, 2nd Stage, Banashankari, ...   
1         Spice Elephant  2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...   
2        San Churro Cafe  1112, Next to KIMS Medical College, 17th Cross...   
3  Addhuri Udupi Bhojana  1st Floor, Annakuteera, 3rd Stage, Banashankar...   
4          Grand Village  10, 3rd Floor, Lakshmi Associates, Gandhi Baza...   

       location rate            rest_type                        cuisines  \
0  Banashankari  4.1        Casual Dining  North Indian, Mughlai, Chinese   
1  Banashankari  4.1        Casual Dining     Chinese, North Indian, Thai   
2  Banashankari  3.8  Cafe, Casual Dining          Cafe, Mexican, Italian   
3  Banashankari  3.7          Quick Bites      South Indian, North Indian   
4  Basavanagudi  3.8        Casual Dining        North Indian, Rajasthani   


In [ ]:
# --- NEW CELL: Filter for Dine-In Restaurants ---

print("Filtering for dine-in restaurants...")
print(f"Original number of restaurants: {len(df)}")

# The '~' symbol means NOT. This line keeps all rows where 'rest_type'
# does NOT contain the word 'Delivery' (case-insensitive).
df = df[~df['rest_type'].str.contains('Delivery', case=False, na=False)]

print(f"Number of restaurants after removing delivery-only places: {len(df)}")
print("✅ Successfully filtered the dataset for dine-in options.")

Filtering for dine-in restaurants...
Original number of restaurants: 51717
Number of restaurants after removing delivery-only places: 47076
✅ Successfully filtered the dataset for dine-in options.


In [ ]:
# --- NEW CELL: Data Type Correction ---
# This cell will fix the data type of the cost column.

# Check if the 'approx_cost(for two people)' column exists
if 'approx_cost(for two people)' in df.columns:
    print("Found 'approx_cost(for two people)' column. Converting it to a numeric type...")
    # The 'coerce' option will turn any values that can't be converted into a 'NaN' (Not a Number)
    df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'], errors='coerce')

    # It's also good practice to drop any rows where the conversion failed
    df.dropna(subset=['approx_cost(for two people)'], inplace=True)

    print("✅ Successfully converted the column to a numeric type.")
else:
    print("❌ ERROR: The column 'approx_cost(for two people)' was not found in your refined file.")

Found 'approx_cost(for two people)' column. Converting it to a numeric type...
✅ Successfully converted the column to a numeric type.


In [ ]:
# This will download the pre-trained model from Hugging Face.
print("Loading the sentiment analysis model...")
sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
print("✅ AI Model loaded successfully!")

Loading the sentiment analysis model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


✅ AI Model loaded successfully!


In [ ]:
# This dictionary maps a detected mood to a list of suitable cuisine types.
mood_to_cuisine_map = {
    'positive': [
        'Italian', 'Pizza', 'Continental', 'Mexican',
        'Mughlai', 'Cafe', 'Desserts', 'North Indian'
    ],
    'negative': [
        'Bakery', 'Desserts', 'Ice Cream', 'Street Food',
        'Biryani', 'North Indian', 'Chinese', 'Fast Food'
    ],
    'neutral': [
        'South Indian', 'Chinese', 'Healthy Food',
        'Beverages', 'Juices', 'Sandwich', 'Tea'
    ]
}
print("✅ Mood-to-Cuisine map is ready!")

✅ Mood-to-Cuisine map is ready!


In [ ]:
def suggest_restaurants(user_input, location, budget=1500, top_n=5):
    """
    Suggests top N restaurants based on user's mood, location, and budget.
    """
    # 1. Get sentiment
    prediction = sentiment_analyzer(user_input)
    label = prediction[0]['label']

    # 2. Convert label to mood
    if label == 'LABEL_2':
        mood = 'positive'
    elif label == 'LABEL_0':
        mood = 'negative'
    else:
        mood = 'neutral'
    print(f"✅ Detected Mood: {mood.capitalize()}")

    # 3. Get suggested cuisines from our map
    suggested_cuisines = mood_to_cuisine_map[mood]
    print(f"✅ Looking for cuisines like: {', '.join(suggested_cuisines)}")

    # 4. Filter the DataFrame
    # Filter by the specified location (case-insensitive)
    location_df = df[df['location'].str.lower() == location.lower()].copy()
    if location_df.empty:
        return f"😕 Sorry, I don't have data for '{location}'. Try another location from your dataset."

    # Filter by cuisines (using a regular expression to match any cuisine in our list)
    cuisine_regex = '|'.join(suggested_cuisines)
    cuisine_mask = location_df['cuisines'].str.contains(cuisine_regex, case=False, na=False)
    filtered_df = location_df[cuisine_mask]

    # Filter by budget
    final_df = filtered_df[filtered_df['approx_cost(for two people)'] <= budget]

    # 5. Check if we found any results
    if final_df.empty:
        return "😕 Couldn't find any matching restaurants. Try a higher budget or a different location."

    # 6. Sort by rating (highest first) and return the top N results
    recommendations = final_df.sort_values(by='rate', ascending=False).head(top_n)
    return recommendations

print("✅ Recommendation function 'suggest_restaurants' is now ready to use!")

✅ Recommendation function 'suggest_restaurants' is now ready to use!


In [ ]:
# --- You can change these values to test the recommender! ---
user_text = "i am not feeling well"
user_location = "BTM"  # Make sure this location exists in your dataset
user_budget = 1200
# -------------------------------------------------------------

print(f"\nSearching for restaurants in '{user_location}' based on your mood...")
print("-" * 70)

# Call the function and get recommendations
recommendations = suggest_restaurants(user_text, user_location, user_budget)

# Print the final result
print("-" * 70)
print(recommendations)


Searching for restaurants in 'BTM' based on your mood...
----------------------------------------------------------------------
✅ Detected Mood: Negative
✅ Looking for cuisines like: Bakery, Desserts, Ice Cream, Street Food, Biryani, North Indian, Chinese, Fast Food
----------------------------------------------------------------------
                          name  \
30805             Cafe Longing   
31919  Rayalaseema Abhiruchulu   
9717                 Kitchen24   
9702                Desi Aahar   
31782               Stop N Joy   

                                                 address location rate  \
30805  119, 6th Cross, 29th Main Road, 2nd Stage, BTM...      BTM  NEW   
31919  18, 10th Cross Road, Keb Layout, Narayan Gowda...      BTM  NEW   
9717   14, Patel Medicals, 7th Main, BTM 2nd Stage, B...      BTM  NEW   
9702   44, 1st Floor, 100 Feet Ring Road, Above Vijay...      BTM  NEW   
31782  72, Jaffer Sadiq Residency, 10th Main, 1st Sta...      BTM  NEW   

         re